In [1]:
# import numpy as np 
# import pandas as pd 
# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input/uc-merced-land-use-dataset/UCMerced_LandUse/Images'):
        
# #     for filename in filenames:
# #         print(os.path.join(dirname, filename))
        

In [2]:
#Load the image data from the directory
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms as T
from torch.utils.data import DataLoader, random_split

transform = T.Compose([T.Resize((256,256)),
                       T.ToTensor(), 
                       T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                      ])

dataset = datasets.ImageFolder(root='/Users/khevinjugessur/Documents/ENEL525/Project/UCMerced_LandUse/Images', transform=transform)

train_dataset, test_dataset = random_split(dataset, [0.8,0.2])

train_dl = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=32, shuffle=True)


for img, label in train_dataset:
    print(img.shape)
    break

torch.Size([3, 256, 256])


In [3]:
if torch.cuda.is_available():
    device=torch.device(type="cuda",index=0)
else:
    device=torch.device(type="cpu",index=0)

print(device)

cpu:0


In [4]:
next(iter(train_dl))

[tensor([[[[ 0.4902,  0.5529,  0.5922,  ...,  0.0353,  0.0275,  0.0510],
           [ 0.5137,  0.5765,  0.5843,  ...,  0.0667,  0.0431,  0.0353],
           [ 0.5373,  0.5765,  0.5843,  ...,  0.0588,  0.0510,  0.0275],
           ...,
           [ 0.6549,  0.6706,  0.0118,  ..., -0.0118,  0.0039,  0.0039],
           [ 0.6784,  0.7176,  0.0275,  ..., -0.0196,  0.0275, -0.0039],
           [ 0.6941,  0.7333,  0.0118,  ...,  0.0118, -0.0275,  0.0431]],
 
          [[ 0.2471,  0.3255,  0.3647,  ..., -0.1059, -0.1216, -0.1059],
           [ 0.2784,  0.3490,  0.3725,  ..., -0.0902, -0.1216, -0.1216],
           [ 0.3020,  0.3490,  0.3725,  ..., -0.0980, -0.1059, -0.1216],
           ...,
           [ 0.5137,  0.4667, -0.1137,  ..., -0.1608, -0.1529, -0.1529],
           [ 0.5373,  0.5216, -0.0980,  ..., -0.1686, -0.1294, -0.1608],
           [ 0.5608,  0.5451, -0.0980,  ..., -0.1373, -0.1686, -0.1137]],
 
          [[ 0.0353,  0.0980,  0.1294,  ..., -0.2000, -0.2078, -0.1922],
           [ 

In [5]:
import torch.nn as nn

class DRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu=nn.ReLU()
        self.conv1=nn.Conv2d(3, 8, kernel_size=(3,3), stride=1, padding=0)
        self.bn1=nn.BatchNorm2d(8)
        self.mp1=nn.MaxPool2d(kernel_size=(2,2),stride=2,padding=0)

        self.conv2=nn.Conv2d(8, 16, kernel_size=(3,3), stride=1, padding=0)
        self.bn2=nn.BatchNorm2d(16)

        self.conv3=nn.Conv2d(16, 32, kernel_size=(3,3), stride=1, padding=0)
        self.bn3=nn.BatchNorm2d(32)

        self.conv4=nn.Conv2d(16, 16, kernel_size=(3,3), stride=1, padding=0)
        self.bn4=nn.BatchNorm2d(16)
        

        self.flatten=nn.Flatten()
        
        #UC Merced has img size 256x256
        self.lin1=nn.Linear(in_features=61504, out_features=21)
        self.bn5=nn.BatchNorm1d(num_features=21)

    def forward(self,x):
        x=self.conv1(x) #3 -> 8, 254x254x8
        x=self.relu(self.bn1(x))
        x=self.conv2(x)#8 -> 16 252x252x16
        x=self.relu(self.bn2(x))
        x=self.mp1(x)# 126x126x16
        x=self.conv4(x)#16->16 126x126x16
        x=self.relu(self.bn4(x))
        x=self.mp1(x)#63x63x16
        x=self.relu(x)
        
        x=self.flatten(x)
        x=self.lin1(x)
        output=self.bn5(x)
        
        return output



In [6]:
batch_size=32
def train_one_epoch(dataloader, model,loss_fn, optimizer):
    model.train()
    track_loss=0
    num_correct=0
    for i, (imgs, labels) in enumerate(dataloader):
        imgs=imgs.to(device)
        labels=labels.to(device)
        pred=model(imgs)

        loss=loss_fn(pred,labels)
        track_loss+=loss.item()
        num_correct+=(torch.argmax(pred,dim=1)==labels).type(torch.float).sum().item()

        running_loss=round(track_loss/(i+(imgs.shape[0]/batch_size)),2)
        running_acc=round((num_correct/((i*batch_size+imgs.shape[0])))*100,2)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i%100==0:
            print("Batch:", i+1, "/",len(dataloader), "Running Loss:",running_loss, "Running Accuracy:",running_acc)

    epoch_loss=running_loss
    epoch_acc=running_acc
    return epoch_loss, epoch_acc

def eval(dataloader, model,loss_fn, path):
    model.eval()
    with torch.no_grad():
        for i, imgs in enumerate(dataloader):
            imgs=imgs.to(device)
            pred=model(imgs)

            pred=torch.argmax(pred,dim=1).type(torch.int).cpu()
            print(pred)


In [7]:
model=DRNN()
model=model.to(device)
loss_fn=nn.CrossEntropyLoss()
lr=0.001
#optimizer=torch.optim.SGD(params=model.parameters(), lr=lr)
optimizer=torch.optim.Adam(params=model.parameters(), lr=lr)
n_epochs=10

for i in range(n_epochs):
    print("Epoch No:",i+1)
    train_epoch_loss, train_epoch_acc=train_one_epoch(train_dl,model,loss_fn,optimizer)
    print("Training:", "Epoch Loss:", train_epoch_loss, "Epoch Accuracy:", train_epoch_acc)
    print("--------------------------------------------------")


Epoch No: 1
Batch: 1 / 53 Running Loss: 3.51 Running Accuracy: 6.25
Training: Epoch Loss: 2.5 Epoch Accuracy: 29.17
--------------------------------------------------
Epoch No: 2
Batch: 1 / 53 Running Loss: 1.95 Running Accuracy: 50.0
Training: Epoch Loss: 1.63 Epoch Accuracy: 65.83
--------------------------------------------------
Epoch No: 3
Batch: 1 / 53 Running Loss: 0.95 Running Accuracy: 93.75
Training: Epoch Loss: 1.01 Epoch Accuracy: 86.79
--------------------------------------------------
Epoch No: 4
Batch: 1 / 53 Running Loss: 0.77 Running Accuracy: 90.62
Training: Epoch Loss: 0.7 Epoch Accuracy: 94.58
--------------------------------------------------
Epoch No: 5
Batch: 1 / 53 Running Loss: 0.63 Running Accuracy: 96.88
Training: Epoch Loss: 0.55 Epoch Accuracy: 96.67
--------------------------------------------------
Epoch No: 6
Batch: 1 / 53 Running Loss: 0.46 Running Accuracy: 96.88
Training: Epoch Loss: 0.48 Epoch Accuracy: 97.8
------------------------------------------

In [8]:
def evaluate_model(test_dl, model, loss_fn):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  # Disable gradient calculations for evaluation
        for inputs, labels in test_dl:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to the appropriate device
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_fn(outputs, labels)
            test_loss += loss.item()
            
            # Get predictions
            _, preds = torch.max(outputs, 1)
            correct_predictions += (preds == labels).sum().item()
            total_samples += labels.size(0)
    
    # Calculate average loss and accuracy
    avg_loss = test_loss / len(test_dl)
    accuracy = correct_predictions / total_samples * 100  # Accuracy in percentage
    
    return avg_loss, accuracy

# Evaluate the model on the test dataset
test_loss, test_accuracy = evaluate_model(test_dl, model, loss_fn)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


Test Loss: 2.0772, Test Accuracy: 42.62%
